<a href="https://colab.research.google.com/github/amarsinghen/landmark-detection-kaggle/blob/master/google_landmark_detection_data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting the tensorflow to version 2.0

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Imports
All the imports for the project

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf
from PIL import Image
import imageio
import logging as log
import tarfile
import os

In [26]:
tf.__version__

'2.1.0-rc1'

Setting the log level

In [0]:
log.basicConfig(level=log.DEBUG)

Downloading Training set Tar files to Mounted google drive. This is a one time task

In [0]:
# for i in range(0,1):
#   if i<10:
#     download_url = "https://s3.amazonaws.com/google-landmark/train/images_00"+str(i)+".tar"
#     save_url = "drive/My\ Drive/datasets/landmark_detection/train/images_00"+str(i)+".tar"
#   elif (i<100 and i>9):
#     download_url = "https://s3.amazonaws.com/google-landmark/train/images_0"+str(i)+".tar"
#     save_url = "drive/My\ Drive/datasets/landmark_detection//train/images_0"+str(i)+".tar"
#   else:
#     download_url = "https://s3.amazonaws.com/google-landmark/train/images_"+str(i)+".tar"
#     save_url = "drive/My\ Drive/datasets/landmark_detection//train/images_"+str(i)+".tar"
#   print(download_url)
#   !wget --no-check-certificate {download_url} -O {save_url}

https://s3.amazonaws.com/google-landmark/train/images_000.tar
--2020-01-15 16:07:45--  https://s3.amazonaws.com/google-landmark/train/images_000.tar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.24.190
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.24.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067018752 (1018M) [application/x-tar]
Saving to: ‘drive/My Drive/datasets/landmark_detection/train/images_000.tar’

drive/My Drive/data 100%[===================>]   1018M  48.9MB/s    in 21s     

2020-01-15 16:08:06 (48.8 MB/s) - ‘drive/My Drive/datasets/landmark_detection/train/images_000.tar’ saved [1067018752/1067018752]



Downloading test set Tar files to Mounted google drive. This is a one time task

In [0]:
# for i in range(0,20):
#   if i<10:
#     download_url = "https://s3.amazonaws.com/google-landmark/test/images_00"+str(i)+".tar"
#     save_url = "drive/My\ Drive/datasets/landmark_detection/test/images_00"+str(i)+".tar"
#   else:
#     download_url = "https://s3.amazonaws.com/google-landmark/test/images_0"+str(i)+".tar"
#     save_url = "drive/My\ Drive/datasets/landmark_detection/test/images_0"+str(i)+".tar"
#   print(download_url)
#   !wget --no-check-certificate {download_url} -O {save_url}

In [0]:
tf.keras.backend.clear_session()

In [0]:
# !ls -l "drive/My Drive/datasets/landmark_detection/train" | wc -l
!find "drive/My Drive/datasets/landmark_detection/train" -type f | wc -l

36635


Extracting the train images from all the 500 tar files to train folder. This is also a 1 time step.

In [0]:
# train_extract_location = "drive/My Drive/datasets/landmark_detection/train"
# for i in range(0,50):
#   if i<10:
#     train_tar_location = "drive/My Drive/datasets/landmark_detection/train/images_00"+str(i)+".tar"
#   elif (i>9 and i<100):
#     train_tar_location = "drive/My Drive/datasets/landmark_detection/train/images_0"+str(i)+".tar"
#   else:
#     train_tar_location = "drive/My Drive/datasets/landmark_detection/train/images_"+str(i)+".tar"
#   print(train_tar_location)
#   my_tar = tarfile.open(train_tar_location)
#   my_tar.extractall(train_extract_location)
#   my_tar.close()

Extracting the test images from all the 20 tar files in test folder. This is also a 1 time step.

In [0]:

# extract_location = "drive/My Drive/datasets/landmark_detection/test"
# for i in range(0,20):
#   if i<10:
#     tar_location = "drive/My Drive/datasets/landmark_detection/test/images_00"+str(i)+".tar"
#   else:
#     tar_location = "drive/My Drive/datasets/landmark_detection/test/images_0"+str(i)+".tar"
#   print(tar_location)
#   my_tar = tarfile.open(tar_location)
#   my_tar.extractall(extract_location)
#   my_tar.close()

In [0]:
# !rm -r "drive/My Drive/datasets/landmark_detection/train/1"

### Data Download
Assigning Training and Test Data file URLS to variables

In [0]:
TRAIN_DATA_URL = "https://s3.amazonaws.com/google-landmark/metadata/train.csv"
TRAIN_ATTRIBUTION_DATA_URL = "https://s3.amazonaws.com/google-landmark/metadata/train_attribution.csv"
TRAIN_LABEL_TO_CATEGORY_DATA_URL = "https://s3.amazonaws.com/google-landmark/metadata/train_label_to_category.csv"
TRAIN_IMAGES_DATA_TAR_URL = "https://s3.amazonaws.com/google-landmark/train/images_001.tar"

TEST_DATA_CSV_URL = "https://s3.amazonaws.com/google-landmark/metadata/test.csv"
TEST_DATA_RECOGNITION_SOLUTION_V2_URL = "https://s3.amazonaws.com/google-landmark/ground_truth/recognition_solution_v2.1.csv"
TEST_DATA_RETRIEVAL_SOLUTION_V2_URL = "https://s3.amazonaws.com/google-landmark/ground_truth/retrieval_solution_v2.1.csv"
TEST_IMAGES_DATA_TAR_URL = "https://s3.amazonaws.com/google-landmark/test/images_000.tar"

Downloading the test and training data (csv) files. These csv files contain the urls and landmark classification information. The variables below are of type string that holds the file locations on local server. The overall image dataset is very large, ~4 million images for training and ~200k for test

In [6]:
train_file_csv = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
train_attribution_csv= tf.keras.utils.get_file("train_attribution.csv", TRAIN_ATTRIBUTION_DATA_URL)
train_label_to_category_csv = tf.keras.utils.get_file("train_label_to_category.csv",TRAIN_LABEL_TO_CATEGORY_DATA_URL)

15155200/15153105 [==============================] - 0s 0us/step


In [0]:
log.debug("The type of variables is : " + str(type(train_file_csv)))

In [8]:
test_file_csv = tf.keras.utils.get_file("test.csv", TEST_DATA_CSV_URL)
test_image_recognition_solution_csv= tf.keras.utils.get_file("test_images_recognition_solution.csv", TEST_DATA_RECOGNITION_SOLUTION_V2_URL)
test_image_retrieval_solution_csv = tf.keras.utils.get_file("test_images_retrieval_solution.csv",TEST_DATA_RETRIEVAL_SOLUTION_V2_URL)

3784704/3780702 [==============================] - 0s 0us/step


### Converting to pandas data frame
Loading training and test data from csv files and converting to pandas dataframe

In [0]:
train_file_csv_df = pd.read_csv(train_file_csv)
train_attribution_csv_df = pd.read_csv(train_attribution_csv)
train_label_to_category_csv_df = pd.read_csv(train_label_to_category_csv)

In [0]:
test_file_csv_df = pd.read_csv(test_file_csv)
test_image_recognition_solution_csv_df = pd.read_csv(test_image_recognition_solution_csv)
test_image_retrieval_solution_csv_csv_df = pd.read_csv(test_image_retrieval_solution_csv)

Create a Dictionary of image ids to landmark_ids

- Checking for **null values**

In [31]:
train_file_csv_df[train_file_csv_df.id.isnull() | train_file_csv_df.url.isnull() | train_file_csv_df.landmark_id.isnull()].count()

id             0
url            0
landmark_id    0
dtype: int64

* The **top 20** landmarks with highest number of images in the training set

In [32]:
train_file_csv_df.landmark_id.value_counts()[:20]

138982    10247
62798      4333
177870     3327
176528     3243
192931     2627
126637     2589
83144      2351
171772     2268
20409      2248
151942     1727
84689      1721
139894     1717
62074      1637
10618      1539
45428      1513
41808      1509
139706     1509
60532      1447
161902     1424
194914     1399
Name: landmark_id, dtype: int64

In [0]:
landmarks_greater_than_5_df = train_file_csv_df.landmark_id.value_counts().reset_index(name="count").query('count>5')
log.debug("Sample of record in the train_file_csv_df dataframe : \n" + str(train_file_csv_df.head(1)))
log.debug("Total number of images in the training set : " + str(train_file_csv_df['url'].count()))
log.debug("Total number of unique landmark_ids in the training dataset : " + str(train_file_csv_df.landmark_id.value_counts()
                                                                                 .reset_index(name="count")["index"].count()))
log.debug("Total number of landmarks with greater than 5 images in the dataset : " + str(landmarks_greater_than_5_df["index"].count()))

* Filtering training set to only inlcude images count of greater than 5 for landmarks

In [0]:
filtered_train_greater_than_5_df = train_file_csv_df[train_file_csv_df.landmark_id.isin(landmarks_greater_than_5_df['index'])]
log.debug("Total number of images for landmarks with image count of greater than 5 in the training set after filtering  : " + str(filtered_train_greater_than_5_df["id"].count()))
log.debug(filtered_train_greater_than_5_df.head(1))
log.debug(filtered_train_greater_than_5_df.iloc[1]['url'])

Storing the filtered dataframe to a csv file for home computer use

In [0]:
filtered_df_file_name = "drive/My Drive/datasets/landmark_detection/train/filtered_train_greater_than_5_df.csv"
filtered_train_greater_than_5_df.to_csv(filtered_df_file_name, encoding='utf-8', index=False)

In [62]:
print(filtered_train_greater_than_5_df["id"].count())
filtered_df_csv = pd.read_csv(filtered_df_file_name)
print(filtered_df_csv["id"].count())

3920020
3920020


Moving the files/images to appropriate train and test directories with labeled folders. Structure of the folders is : train->landmark_labeled_folder->actual_image

In [18]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)             
    return allFiles

dirName = "drive/My Drive/datasets/landmark_detection/train/0"

listOfImages = getListOfFiles(dirName)
listOfImages[:10]

['drive/My Drive/datasets/landmark_detection/train/0/0/0/0000059611c7d079.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/0000070506c174cc.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/000008ae30de967e.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/000014b1f770f640.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/000015f76534add3.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/00001ae42cd00356.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/00001b2ba2c69ac5.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/00001d45edef6c9e.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/000020dfc079c013.jpg',
 'drive/My Drive/datasets/landmark_detection/train/0/0/0/000022ce0b9d8966.jpg']

In [37]:
filtered_train_greater_than_5_df[:1]

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820


In [56]:
train_file_csv_df[train_file_csv_df.id=='0000070506c174cc'].values

array([['0000070506c174cc',
        'http://upload.wikimedia.org/wikipedia/commons/0/06/Grabst%C3%A4tte_Stra%C3%9Fe_des_17_Juni_%28Tierg%29_Sowjetisches_Ehrenmal.jpg',
        149519]], dtype=object)

In [59]:
import shutil

for filepath in listOfImages[:1000]:
  file_name = filepath.split("/")[-1].split(".")[0]
  landmark_id = filtered_train_greater_than_5_df[filtered_train_greater_than_5_df.id==file_name]
  if (len(landmark_id)!=0):
    landmark_id_folder_name = str(landmark_id['landmark_id'].values[0])
    local_location = str("/tmp/train/" + str(landmark_id_folder_name) + "/" + str(file_name) + ".jpg")
    if not os.path.isdir(local_location.split(str(file_name))[0]):
        os.makedirs(local_location.split(str(file_name))[0])
    # print(len(landmark_id))
    # print(str(landmark_id['landmark_id'].values[0]))
    # print(file_name)
    shutil.copy2(filepath, local_location)
    break

1
149519
0000070506c174cc


In [0]:
!pwd

/content


Download all the images via url. Most likely would not use this piece of code.

In [0]:
# import urllib.request
# from urllib.error import *
# import os
# # for row in filtered_train_between_50and100_df.iterrows():
# for index, row in filtered_train_between_50and100_df.iterrows():
#   # print(row['id'])
#   # print(row['url'])
#   # print(row['landmark_id'])
#   local_location = str("/tmp/" + str(row['landmark_id']) + "/" + str(row['id']) + ".jpg")
#   # print(local_location)
#   if not os.path.isdir(local_location.split(row['id'])[0]):
#         os.makedirs(local_location.split(row['id'])[0])
#   try:
#     urllib.request.urlretrieve(row['url'], local_location)
#   except FileNotFoundError as err:
#     print(err)   # something wrong with local path
#   except HTTPError as err:
#     print(err)  # something wrong with url
#     print(row['url'])
#   except ContentTooShortError as err:
#     print(err)  # something wrong with url
#     print(row['url'])

In [0]:
log.debug("Sample of record in the test_file_csv_df dataframe : \n" + str(test_file_csv_df.head(3)))
log.debug("Total number of images in the test set : " + str(test_file_csv_df['id'].count()))
log.debug("Total number of unique landmark_ids in the test dataset : " + str(test_file_csv_df.id.value_counts()
                                                                                 .reset_index(name="count")["index"].count()))

View a few pictures. Configuring matplot parameters

In [0]:
%matplotlib inline

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_path = "/tmp/100157/9cd60ce98c029534.jpg"

fig = plt.gcf()
img = mpimg.imread(image_path)
plt.imshow(img)
plt.show()

Build a tensorflow model

In [0]:
import tensorflow as tf
model=tf.keras.models.Sequential([
                                  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(400,400,3)),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                  tf.keras.layers.MaxPooling2D(2,2),
                                  tf.keras.layers.Flatten(),
                                  tf.keras.layers.Dense(512, activation='relu'),
                                  tf.keras.layers.Dense(filtered_train_between_50and100_df["id"].count(), activation='softmax')
])
model.summary()

Compiling the model

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

Data preprocessing with ImageDataGenerator

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255)
#flow training images in batches of 128
train_generator = train_datagen.flow_from_directory(
    '/tmp/',
    target_size=(400,400),
    batch_size=128,
    class_mode='categorical'
)
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs = 15,
    verbose = 1
)

Evaluating Accuracy and Loss

In [0]:
acc = history.history['acc']
loss = history.history['loss']
epochs = range(len(acc))

plt.plot(epochs, acc)
plt.title('Training accuracy')
plt.figure()

plt.plot(epochs, loss)
plt.title('Training Loss')
plt.figure()